In [ ]:
%pip install pandas
%pip install pydmd
%pip install scikit-learn
%pip install matplotlib
%pip install seaborn
%pip install keras_tuner
%pip install keras
%pip install numpy

In [ ]:
import sys

base_path = '/home/yurid/git/tcc'
sys.path.insert(0, base_path)

In [ ]:
from math import sqrt

import keras
import numpy as np
import pandas as pd
from keras import layers
from keras.callbacks import EarlyStopping
from keras_tuner.tuners import RandomSearch
from matplotlib import pyplot
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Bidirectional, Dropout, Activation, Dense, LSTM, InputLayer
from keras.callbacks import EarlyStopping
from keras.layers import RepeatVector
from keras.layers import TimeDistributed



from src.filler import dmd_filler, interpolate_filler, fill_df
from src.dataframe import read_data, windowing, split_data, show_error_metrics
from src.scaler import Scaler
from src.model import train_model, hp_search
from src.model import hp_search, build_model

In [ ]:
df_temp = pd.read_csv(
    f'{base_path}/data/daily_juiz de fora.csv',
    sep=',',
    parse_dates=['Timestamp'],
    index_col=['Timestamp'])

init_index = lambda _df: _df[_df.RADIATION != 0].index[0]

df = df_temp[['RADIATION', 'TEMP', 'HUMIDITY_h']] \
    .resample('D') \
    .agg({'RADIATION': np.sum})

df = df.loc[df.index >= init_index(df)].replace(0, np.nan)
print(df.isna().sum())
df

In [ ]:
df = fill_df(df, interpolate_filler)
print(df.isna().sum())

In [ ]:
scaler = Scaler(df)
df_scaled = scaler.get_dataframe_scaled()
df_scaled

In [ ]:
X, Y = windowing(df_scaled, 3, 1)

In [ ]:
for i in range(len(X)):
    print(X[i], Y[i])

In [ ]:
train_x, train_y, test_x, test_y = split_data(X, Y, length=len(df_scaled), ratio=0.9)

In [ ]:
print(train_x.shape[1], train_x.shape[2])

In [ ]:
model = Sequential([
    LSTM(units=200, input_shape=(train_x.shape[1], train_x.shape[2])),
    Dense(1, activation='sigmoid')
])

model.compile(loss='mse', optimizer=keras.optimizers.legacy.Adam(learning_rate=0.01))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=15)
history = model.fit(train_x,
                     train_y,
                     epochs=200,
                     batch_size=16,
                     validation_split=0.3,
                     verbose=1,
                     callbacks=[es])
# plot history
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

In [ ]:
predict = model.predict(test_x)
show_error_metrics(test_y.reshape(-1,1), predict)

In [ ]:
range_plot = -1

pyplot.plot(test_y.reshape(-1,1)[0:range_plot], color='blue', label='real')
pyplot.plot(predict[0:range_plot], color='red', label='incompleto')

show_error_metrics(test_y.reshape(-1,1), predict)

pyplot.show()